In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

In [2]:
import os

input_dir = '/kaggle/input'
print("Contents of /kaggle/input:")
for item in os.listdir(input_dir):
    print(item)

if 'um-game-playing-strength-of-mcts-variants' in os.listdir(input_dir):
    print("\nContents of /kaggle/input/um-game-playing-strength-of-mcts-variants:")
    for item in os.listdir(os.path.join(input_dir, 'um-game-playing-strength-of-mcts-variants')):
        print(item)
else:
    print("\num-game-playing-strength-of-mcts-variants directory not found.")

Contents of /kaggle/input:
um-game-playing-strength-of-mcts-variants

Contents of /kaggle/input/um-game-playing-strength-of-mcts-variants:
sample_submission.csv
train.csv
test.csv
concepts.csv
kaggle_evaluation


In [3]:
import os
import sys
import polars as pl
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from pathlib import Path

# Try to import the Kaggle evaluation module, but don't fail if it's not available
try:
    import kaggle_evaluation.mcts_inference_server
    KAGGLE_EVALUATION_AVAILABLE = True
except ImportError:
    KAGGLE_EVALUATION_AVAILABLE = False
    print("Warning: kaggle_evaluation module not found. Running in development mode.")

# Set the path to your data files
comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')
target = 'utility_agent1'

def engineer_features(df):
    # Implement your feature engineering here
    # This is a simplified version; you may want to add more complex feature engineering
    
    # Example: Create some interaction features
    numeric_cols = df.select(pl.col(pl.Float64)).columns
    for i, col1 in enumerate(numeric_cols):
        for col2 in numeric_cols[i+1:]:
            df = df.with_columns((pl.col(col1) * pl.col(col2)).alias(f"{col1}_{col2}_interaction"))
    
    return df

def train_model():
    global obj_cols, preprocessor, rf

    train = pl.read_csv(comp_path / 'train.csv')
    y_train = train[target]

    cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target, 'Id']
    train = train.drop(cols_to_drop)
    
    obj_cols = train.select(pl.col(pl.String)).columns
    num_cols = train.select(pl.col(pl.Float64)).columns
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), obj_cols)
        ])
    
    X_train = preprocessor.fit_transform(train.to_pandas())
    
    rf = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
    rf.fit(X_train, y_train)

counter = 0
def predict(test, submission):
    global counter
    if counter == 0:
        train_model()
    counter += 1
    
    test = test.drop(['Id'])
    X_test = preprocessor.transform(test.to_pandas())
    predictions = rf.predict(X_test)
    
    return submission.with_columns(pl.Series(target, predictions))
# Main execution
if KAGGLE_EVALUATION_AVAILABLE:
    # Calling the gateway server
    inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            (
                '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
                '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
            )
        )
else:
    # Development mode: Train the model and make predictions on a sample of the test data
    train_model()
    
    # Load a sample of the test data (adjust the number of rows as needed)
    test_sample = pl.read_csv(comp_path / 'test.csv').sample(n=1000, seed=42)
    submission_sample = pl.read_csv(comp_path / 'sample_submission.csv').sample(n=1000, seed=42)
    
    # Make predictions
    result = predict(test_sample, submission_sample)
    
    print("Sample predictions:")
    print(result.head())

    print("\nModel training and sample prediction completed successfully.")